# Algoritmos de optimización - Seminario

Nombre y Apellidos: Álvaro Navarro Guzmán

Url: https://github.com/alvaronavarroguz/03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO

Problema:

#### Organizar los horarios de partidos de La Liga

Descripción del problema:

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos
de liga de de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios que maximice la
audiencia 

(*) La respuesta es obligatoria

In [1]:
import numpy as np
import random
import pandas as pd

1(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.

Respuesta

- Sin repetir horario, sabiendo que tenemos 10 partidos jugados en 10 horarios distintos: para el primer partido tendríamos 10 posibilidades, para el segundo 9, para el tercero 8...de modo que tendríamos 10! prosibilidades distintas.

- Pudiendo repetir horario, tendríamos 10 partidos jugados en 10 horarios distintos = 10^10 posibilidades
    

2.Modelo para el espacio de soluciones

(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)

Respuesta

Para describir los datos de entrada trabajaremos con DataFrames, por la posibilidad de asignar horarios y número de espectadores a los varios partidos que se disputan en una jornada y todo a través de un índice común. De esta forma se nos facilita el acceso a cada partido


In [2]:
jornada=pd.DataFrame()
jornada=jornada.append({'Partidos':'Barcelona-Real Madrid','Categoria':'AA'} , ignore_index=True)
jornada=jornada.append({'Partidos':'Atlético-Getafe','Categoria':'AB' }, ignore_index=True)
jornada=jornada.append({'Partidos':'Alavés-Sevilla','Categoria':'BB'}, ignore_index=True)
jornada=jornada.append({'Partidos':'Valencia-Betis','Categoria':'AB' }, ignore_index=True)
jornada=jornada.append({'Partidos':'R.Sociedad-Eibar','Categoria':'BB' }, ignore_index=True)
jornada=jornada.append({'Partidos':'Espanyol-Athletic','Categoria':'BB' }, ignore_index=True)
jornada=jornada.append({'Partidos':'Leganés-Girona','Categoria':'CB' }, ignore_index=True)
jornada=jornada.append({'Partidos':'Levante-Valladolid','Categoria':'BC'}, ignore_index=True )
jornada=jornada.append({'Partidos':'Celta-Villareal','Categoria':'BB'}, ignore_index=True )
jornada=jornada.append({'Partidos':'Rayo Vallecano-Huesca','Categoria':'CC' }, ignore_index=True)
jornada

,Categoria,Partidos
0,AA,Barcelona-Real Madrid
1,AB,Atlético-Getafe
2,BB,Alavés-Sevilla
3,AB,Valencia-Betis
4,BB,R.Sociedad-Eibar
5,BB,Espanyol-Athletic
6,CB,Leganés-Girona
7,BC,Levante-Valladolid
8,BB,Celta-Villareal
9,CC,Rayo Vallecano-Huesca


El número de espectadores en cada franja horaria lo marcaremos con otro DataFrame para poder acceder a los valores a través de los strings 'A', 'B', 'C'

In [3]:
horarios =['Viernes 20', 'Sábado 12', 'Sábado 16', 'Sábado 18', 'Sábado 20',
           'Domingo 12','Domingo 16', 'Domingo 18', 'Domingo 20', 'Lunes 20']
espectadores=pd.DataFrame([[2,1.3, 1], [1.3, 0.9, 0.75], [1,0.75,0.47]], columns=['A', 'B', 'C'], index=['A', 'B', 'C'])
ajuste_espectador=[0.4, 0.55, 0.7, 0.8, 1, 0.45,0.75, 0.85, 1, 0.4]
val={}
val['A']=4
val['B']=2
val['C']=1

3-4.Según el modelo para el espacio de soluciones

(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo pasa por sumar los espectadores de todos los partidos aplicando los ajustes necesarios que se imponen en el enunciado.

sum(espectadores[horario] x espectadores[categroría]-ajuste_por_repeticion) para todos los partidos

Se trata de un problema de maximización, ya que busca tener el mayor número de espectadores posibles para cada jornada de liga

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta


5.(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

### Algoritmo Voraz

Establecemos el orden de los partidos en el que el algoritmo debe descartar las posibilidades, de modo que ordenamos de mayor a menor por índice de espectadores que generan

In [47]:
#Ajustamos los índices de espectadores en función del horario para el caso en que haya varios
#partidos que se juegan a la misma hora.

def mejor_horario(jornada_input, index_input, ajuste_espectador_input, horarios_recuento_input):
    
    #inicializamos
    new_ajuste_espectador=ajuste_espectador_input.copy()
    ajuste_por_repeticion={'reduccion':[], 'Partido Previo':[], 'Partido Actual':[]}

    for k in np.arange(len(new_ajuste_espectador)):
        
        #Tenemos que ajustar los espectadores en caso de que ya haya un partido esa jornada
        if np.isnan(horarios_recuento_input[horarios[k]]) == False:
            
            index_previo=horarios_recuento_input[horarios[k]]
            categoria_actual= jornada_input.at[index_input, 'Categoria'] 
            categoria_previa= jornada_input.at[index_previo, 'Categoria'] 

            #Calculamos el divisor del ajuste
            divisor=val[categoria_actual[0]]+val[categoria_actual[1]]+val[categoria_previa[0]]+val[categoria_previa[1]]
            
            #Calculamos cuántos espectadores tenía el partido ese día como:  categorías * horario
            audiencia_previa=jornada_input.at[index_previo, 'Espectadores']
            
            #Calculamos estos valores en caso de tener que ofrecer información acerca de cuántos espectadores ven cada partido
            #Pero en realidad, para la optimización del problema, no hace falta.
            ajuste_por_repeticion['reduccion'].append(audiencia_previa*((val[categoria_actual[0]]+val[categoria_actual[1]])/divisor))
            ajuste_por_repeticion['Partido Previo'].append(index_previo)
            ajuste_por_repeticion['Partido Actual'].append(index_input)
            
            #Al repartirse los espectadores del mejor partido, este segundo encuentro no aporta ningún espectador más
            #al cómputo total de la jornada de liga.            
            new_ajuste_espectador[horarios.index(horarios[k])]=0
            
    return ajuste_por_repeticion, new_ajuste_espectador

            
#Obtenemos los horarios de los partidos para una jornada determinada    
def calendario_jornada(jornada, horarios, espectadores, ajuste_espectador, val): 
    #incializamos el DF
    jornada["Espectadores"] = np.nan

    #Orden de preferencia de partidos a rellenar franjas horarias con más espectadores.
    ord=['AA', 'AB', 'AC', 'BB','BC', 'CC']#
    
    #Anotar las franjas horarias usadas
    horarios_recuento=dict(zip(horarios, np.empty(len(horarios)) * np.nan))
    
    for i in ord:
        #Comprobamos los partidos que coinciden con el tipo de categoría (i) 
        list_match=list(jornada.index[(jornada['Categoria']==i) | (jornada['Categoria']==i[::-1])])
        #puede ser que no se dispute ningún partido de esa categoria
        if list_match:
            #Puede ser que se den varios partidos con la misma categoría
            for j in list_match:
                #Vemos cuáles son las franjas horarias disponibles y cuál es la audiencia en cada una
                repeticion, new_ajuste_espectador=mejor_horario(jornada, j,ajuste_espectador, horarios_recuento )

                #Elegimos la más alta (voraz)
                index=np.where(new_ajuste_espectador==np.max(new_ajuste_espectador))

                #Completamos el DataFrame para ofrecer toda la información
                jornada.at[j, 'Horario']= horarios[index[0][0]] #Horario del partido
                jornada.at[j, 'Multiplicador por Categoría']= espectadores[i[0]][i[1]]#Espectadores por la categoría del partido
                jornada.at[j, 'Multiplicador por Horario']= new_ajuste_espectador[index[0][0]] #Espectadores por el horario del partido
                jornada.at[j, 'Ajuste por repetición']= False #Comprobar si se repite con algún otro partido
                
                #Calculamos los espectadores totales del partido
                jornada.at[j, 'Espectadores']= jornada.at[j, 'Multiplicador por Categoría']*jornada.at[j, 'Multiplicador por Horario']
                
                #En caso de existir una repetición de horario...
                if new_ajuste_espectador[index[0][0]] == 0:
                    jornada.at[j, 'Ajuste por repetición']= True
                    jornada.at[repeticion['Partido Previo'][index[0][0]], 'Ajuste por repetición']= True
                    
                    #Hacemos un ajuste al número de espectadores de los partidos que coinciden en el mismo horario
                    jornada.at[j, 'Espectadores']=repeticion['reduccion'][index[0][0]]
                    
                    valor_previo=jornada.at[repeticion['Partido Previo'][index[0][0]], 'Espectadores']
                    valor_diff=repeticion['reduccion'][index[0][0]]
                    
                    jornada.at[repeticion['Partido Previo'][index[0][0]], 'Espectadores']=valor_previo-valor_diff
                    
                #Asignamos el horario a un partido para comprobar repeticiones en futuras iteraciones    
                horarios_recuento[horarios[index[0][0]]]=j
    return jornada
     

In [5]:
#Comprobamos
jornada_test =   calendario_jornada(jornada, horarios, espectadores, ajuste_espectador, val)
jornada_test

,Categoria,Partidos,Espectadores,Horario,Multiplicador por Categoría,Multiplicador por Horario,Ajuste por repetición
0,AA,Barcelona-Real Madrid,2.0000,Sábado 20,2.00,1.00,False
1,AB,Atlético-Getafe,1.3000,Domingo 20,1.30,1.00,False
2,BB,Alavés-Sevilla,0.7200,Sábado 18,0.90,0.80,False
3,AB,Valencia-Betis,1.1050,Domingo 18,1.30,0.85,False
4,BB,R.Sociedad-Eibar,0.6750,Domingo 16,0.90,0.75,False
5,BB,Espanyol-Athletic,0.6300,Sábado 16,0.90,0.70,False
6,CB,Leganés-Girona,0.3375,Domingo 12,0.75,0.45,False
7,BC,Levante-Valladolid,0.3000,Viernes 20,0.75,0.40,False
8,BB,Celta-Villareal,0.4950,Sábado 12,0.90,0.55,False
9,CC,Rayo Vallecano-Huesca,0.1880,Lunes 20,0.47,0.40,False


In [6]:
print('El número total de espectadores es: %2.2f Millones'%(jornada['Espectadores'].sum()))

El número total de espectadores es: 7.75 Millones


Mejora el algoritmo de fuerza bruta ya que no comprueba todas las posibles combinaciones del problema. Asocia franjas horarias más vistas con partidos con más espectadores sin llegar a comprobar el resto.

De ahí que el número de instancias en el algoritmo voraz descrito sea mucho menor que el de fuerza bruta.

6.(*)Calcula la complejidad del algoritmo

Respuesta

El algoritmo depende de tres variables: Número de partidos =10 (constante), Combinaciones de categorías (A, B, C)=3! (constante) y del número de franjas horarias=10 (constante)

De esta forma el algoritmo tendría una complejidad de O(1), pero con una constante de multiplicación bastante alta debido al número de operaciones elementales y asignaciones que se llevan a cabo en las dos funciones descritas.

Y se simplificaría mucho más si no admitiese repeticiones de horario (aunque en el fondo, nunca lleguen a usarse al no ser una solución óptima).


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [9]:
lista_equipos={'Barcelona':'A', 'Real Madrid':'A', 'Valencia':'A', 'Atlético':'A', 'Getafe':'B',
               'Alavés':'B', 'Sevilla':'B', 'Betis':'B', 'R.Sociedad':'B', 'Eibar':'B',
               'Espanyol':'B', 'Athletic':'B', 'Girona':'B', 'Levante':'B', 'Celta':'B',
               'Villareal':'B', 'Rayo Vallecano':'C', 'Huesca':'C', 'Valladolid':'C', 'Leganés':'C'}

equipos=list(lista_equipos.keys())
N=len(equipos)
jornada=pd.DataFrame()

for i in range(len(equipos)//2):
    
    #Se eligen dos equipos random de la lista
    pos=random.sample(range(0, N), 2)
    eq1=equipos[pos[0]]
    eq2=equipos[pos[1]]
    #Se añaden a la jornada
    jornada = jornada.append({'Partidos': eq1+'-'+eq2 , 'Categoria':lista_equipos[eq1]+lista_equipos[eq2]}, ignore_index=True)
    
    #Se eliminan de la lista para evitar repeticiones
    equipos.remove(eq1)
    equipos.remove(eq2)
    
    N=N-2
    
print (jornada)
    

  Categoria              Partidos
0        BA     Athletic-Valencia
1        BB        Espanyol-Betis
2        BC    R.Sociedad-Leganés
3        BB          Getafe-Eibar
4        CB    Valladolid-Levante
5        AB      Atlético-Sevilla
6        BB      Villareal-Alavés
7        BC  Celta-Rayo Vallecano
8        CA    Huesca-Real Madrid
9        BA      Girona-Barcelona



Aplica el algoritmo al juego de datos generado

Respuesta

In [10]:
#Comprobamos si funciona bien con el algoritmo descrito
jornada_test =   calendario_jornada(jornada, horarios, espectadores, ajuste_espectador, val)
jornada_test

,Categoria,Partidos,Espectadores,Horario,Multiplicador por Categoría,Multiplicador por Horario,Ajuste por repetición
0,BA,Athletic-Valencia,1.3000,Sábado 20,1.30,1.00,False
1,BB,Espanyol-Betis,0.6750,Domingo 16,0.90,0.75,False
2,BC,R.Sociedad-Leganés,0.3375,Domingo 12,0.75,0.45,False
3,BB,Getafe-Eibar,0.6300,Sábado 16,0.90,0.70,False
4,CB,Valladolid-Levante,0.3000,Viernes 20,0.75,0.40,False
5,AB,Atlético-Sevilla,1.3000,Domingo 20,1.30,1.00,False
6,BB,Villareal-Alavés,0.4950,Sábado 12,0.90,0.55,False
7,BC,Celta-Rayo Vallecano,0.3000,Lunes 20,0.75,0.40,False
8,CA,Huesca-Real Madrid,0.8000,Sábado 18,1.00,0.80,False
9,BA,Girona-Barcelona,1.1050,Domingo 18,1.30,0.85,False


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Considero que para el problema descrito y con las restricciones establecidas lo que se puede hacer es simplificar el código de forma que se reduzca la memoria usada y mejore la legibilidad.

Ahora, si quisiéramos generalizar este problema para otros tipos de deporte (Tenis, Baloncesto...) o reducir el número de franjas horarias de forma que se aumente la probabilidad de repetir franja para distintos partidos, se podría tender hacia métodos heurísticos o de ramificación y poda.

In [11]:
#Comprobamos que en este caso, el algoritmo es capaz de funcionar si eliminamos el 'Sábado 16'
#como opción de franja horaria.
horarios =['Viernes 20', 'Sábado 12', 'Sábado 18', 'Sábado 20',
           'Domingo 12','Domingo 16', 'Domingo 18', 'Domingo 20', 'Lunes 20']
espectadores=pd.DataFrame([[2,1.3, 1], [1.3, 0.9, 0.75], [1,0.75,0.47]], columns=['A', 'B', 'C'], index=['A', 'B', 'C'])
ajuste_espectador=[0.4, 0.55, 0.8, 1, 0.45,0.75, 0.85, 1, 0.4]
val={}
val['A']=4
val['B']=2
val['C']=1

In [48]:
#Comprobamos si funciona bien con el algoritmo descrito
jornada_test2 =   calendario_jornada(jornada, horarios, espectadores, ajuste_espectador, val)
jornada_test2

,Categoria,Partidos,Espectadores,Horario,Multiplicador por Categoría,Multiplicador por Horario,Ajuste por repetición
0,BA,Athletic-Valencia,1.300,Sábado 20,1.30,1.00,False
1,BB,Espanyol-Betis,0.675,Domingo 16,0.90,0.75,False
2,BC,R.Sociedad-Leganés,0.150,Viernes 20,0.75,0.40,True
3,BB,Getafe-Eibar,0.495,Sábado 12,0.90,0.55,False
4,CB,Valladolid-Levante,0.300,Lunes 20,0.75,0.40,False
5,AB,Atlético-Sevilla,1.300,Domingo 20,1.30,1.00,False
6,BB,Villareal-Alavés,0.405,Domingo 12,0.90,0.45,False
7,BC,Celta-Rayo Vallecano,0.150,Viernes 20,0.75,0.00,True
8,CA,Huesca-Real Madrid,0.800,Sábado 18,1.00,0.80,False
9,BA,Girona-Barcelona,1.105,Domingo 18,1.30,0.85,False


Ahí podemos comprobar cómo se reparten espectadores dos partidos jugados a la misma hora